# Imports

In [1]:
%%time
#Imports requisute packages
import os
import time
import numpy
import pickle
import cProfile
import itertools
import matplotlib
import sklearn.svm
import sklearn.tree
import sklearn.metrics
import sklearn.ensemble
import sklearn.preprocessing
import sklearn.learning_curve
import sklearn.model_selection
import sklearn.cross_validation
import sklearn.feature_selection
import sklearn.kernel_approximation
from matplotlib import pyplot as plt

#%jsroot on9
%matplotlib inline
matplotlib.use('Agg')

/usr/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/anaconda3/lib/python3.6/site-packages/sklearn/learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


CPU times: user 637 ms, sys: 64 ms, total: 701 ms
Wall time: 1.29 s


/usr/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
#Converts ROOT tree to numpy array

# %%time
# tChain = rt.TChain('MyAnalysis/MyTree')
# tChain.Add("ntuples/*.root")
# array = root_numpy.tree2array(tChain)
# print 'Total number of entries: ',tChain.GetEntries()


# Function Definitions

In [3]:
%%time
#Takes the converted tree and turns it into an
#n-by-30 array usable by sklearn.
def outputs(array):
    #Only uses events with non-zero luminosity
    goodEvents = array[array['lumi'] != 0]
    ind = numpy.lexsort((goodEvents['lumiId'],goodEvents['runId']))
    events = goodEvents[ind]
    dataset = numpy.empty([len(goodEvents),30])
    target = numpy.empty([len(goodEvents)])
    badOnes = numpy.array([])

    #Fills dataset array with proper features
    for j, event in enumerate(events):
        try:
            dataset[j,0:7] = event['qPFJetPt']
            dataset[j,7:14] = event['qPFJetEta']
            dataset[j,14:21] = event['qPFJetPhi']
            dataset[j,21:28] = event['qNVtx']
            dataset[j,28] = event['crossSection']
            dataset[j,29] = event['lumi']
            target[j] = event['isSig']
        except ValueError:
            badOnes = numpy.append(badOnes,j)
          
    #Takes out corrupt events
    mask = numpy.zeros(len(dataset), dtype=bool)
    mask[badOnes.astype(int)] = True
    mask = ~mask
    dataset = dataset[mask]
    target = target[mask]
       
    return dataset, target

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 17.6 µs


In [4]:
#Initializes and fits BDT classifier
def dTree(xTrain, yTrain, lumiTrain):
    start = time.time()
    classifier = AdaBoostClassifier(DecisionTreeClassifier(max_depth = 50, 
                                    min_samples_split = 50, min_samples_leaf = 1),
                                    algorithm="SAMME.R", n_estimators=200,
                                    learning_rate=1)
# Comment sample_weight if the training is to be done without sample weight.),
    if weights == True:
        classifier = classifier.fit(xTrain, yTrain, sample_weight = lumiTrain) 
    else:
        classifier = classifier.fit(xTrain, yTrain)

    print('It took', time.time()-start, 'seconds for BDT.')
    return classifier

In [5]:
#Function that plots confusion matrix, taken from sklearn website
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    #This function prints and plots the confusion matrix.
    #Normalization can be applied by setting `normalize=True`.     
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, numpy.newaxis]
        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.title(title)
        plt.colorbar()
        tick_marks = numpy.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=45)
        plt.yticks(tick_marks, classes)
        print("Normalized confusion matrix")
    else:
        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.title(title)
        plt.colorbar()
        tick_marks = numpy.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=45)
        plt.yticks(tick_marks, classes)
        print('Confusion matrix, without normalization')

    print(cm)
    
    thresh = cm.max()*.7
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


# Settings and Data Prep

In [6]:
#Pick learning algorithm, crossSection, weights, and random splitting
BDT = True
crossSection = True
weights = False
random = True
odds = False

#Saves dataset for pickling
# dataset, target = outputs(array)
# outFile = open('data.pkl', 'wb')
# pickle.dump(dataset, outFile)
# pickle.dump(target, outFile)
# outFile.close()

#Loads pickled dataset
inFile = open('realData.pkl', 'rb')
dataset = pickle.load(inFile, encoding="latin1")
target = pickle.load(inFile, encoding="latin1")
inFile.close()

In [7]:
#Splits dataset into training and testing sets based on settings
xTrain = []
yTrain = []
xTest = []
yTest = []

if random == True:
    xTrain, xTest, yTrain, yTest = train_test_split(dataset, target, test_size=0.5)

    lumiTrain = xTrain[:,-1]
    lumiTest = xTest[:,-1]
    lumiTrain = numpy.copy(lumiTrain, order='C')
    lumiTest = numpy.copy(lumiTest, order='C')

    if crossSection == True:
        xTrain = xTrain[:,:-1]
        xTest = xTest[:,:-1]
    else:
        xTrain = xTrain[:,:-2]
        xTest = xTest[:,:-2]
elif odds == True:
    lumiTrain = dataset[::2,-1]
    lumiTest = dataset[1::2,-1]
    lumiTrain = numpy.copy(lumiTrain, order='C')
    lumiTest = numpy.copy(lumiTest, order='C')
    
    xTrain = dataset[::2]
    xTest = dataset[1::2]
    yTrain = target[::2]
    yTest = target[1::2]
else:
    n = int(numpy.floor(len(dataset)/5))
    lumiTrain = dataset[:n*4,-1]
    lumiTest = dataset[n*4:,-1]
    lumiTrain = numpy.copy(lumiTrain, order='C')
    lumiTest = numpy.copy(lumiTest, order='C')
    
    if crossSection ==  True:
        xTrain = dataset[:n*4,:-1]
        xTest = dataset[n*4:,:-1]
    else:
        xTrain = dataset[:n*4,:-2]
        xTest = dataset[n*4:,:-2]
        
    yTrain = target[:n*4]
    yTest = target[n*4:]

NameError: name 'train_test_split' is not defined

In [ ]:
#Imports training and testing sets used across models
inFile = open('splits.pkl', 'rb')
xTrain = pickle.load(inFile, encoding="latin1")
xTest = pickle.load(inFile, encoding="latin1")
yTrain = pickle.load(inFile, encoding="latin1")
yTest = pickle.load(inFile, encoding="latin1")
lumiTrain = pickle.load(inFile, encoding="latin1")
lumiTest = pickle.load(inFile, encoding="latin1")
inFile.close()

In [ ]:
#Removes features with variance less than 0.1
sel = VarianceThreshold(threshold = 0.01)
print(xTrain.shape)
sel.fit(xTrain)
indices = sel.get_support()
xTrain = xTrain[:,indices]
print(xTrain.shape)
xTest = xTest[:,indices]
print(indices)

#Removes features based on different metrics

# print(xTrain.shape)
# genSel = GenericUnivariateSelect(chi2, mode = 'k_best', param = 5)
# genSel.fit(xTrain,yTrain)
# indices = genSel.get_support()
# print(indices)
# xTrain = xTrain[:,indices]
# xTest = xTest[:,indices]
# print(xTrain.shape)
# print(xTest.shape)
# mask = numpy.ones(29,dtype=bool)
# print(mask)
# ind = [1, 2, 6, 7, 9, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21]
# mask[ind] = False
# print(mask)
# print(xTrain.shape)
# xTrain = xTrain[:,mask]
# print(xTrain.shape)
# xTest = xTest[:,mask]

In [ ]:
#Scales the data to zero mean and unit variance
scaler = StandardScaler()
scaler.fit(xTrain)
xTrain = scaler.transform(xTrain)
xTest = scaler.transform(xTest)
print(len(yTrain[yTrain==0]))
print(len(yTrain[yTrain==1]))

In [ ]:
%%time
# Grid Search with cross validation

tunedParams = [{'algorithm': ['SAMME', 'SAMME.R']}]
scores = ['precision', 'recall', 'roc_auc', 'f1']
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(AdaBoostClassifier(DecisionTreeClassifier(max_depth = 50, 
                       min_samples_split = 50, min_samples_leaf = 10)),
                       tunedParams, cv=3, n_jobs=-1, scoring='%s' % score)
    clf.fit(xTrain, yTrain)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = yTest, clf.predict(xTest)
    print(classification_report(y_true, y_pred))
    print()

In [ ]:
%%time
#Kernel approximation

# rbfFeature = RBFSampler(gamma = 10, n_components = 10000)
# rbfFeature.fit(xTrain)
# newXTrain = rbfFeature.transform(xTrain)
# newXTest = rbfFeature.transform(xTest)
# nysFeature = Nystroem(gamma = 100, n_components = 1000)
# nysFeature.fit(xTrain)
# newXTrain = nysFeature.transform(xTrain)
# newXTest = nysFeature.transform(xTest)


# Trains and Tests

In [ ]:
%%time
#Calls classifier function
bdtClf = dTree(xTrain, yTrain, lumiTrain)

In [ ]:
#Pickles classifier

# outClfs = open('bdtClf_tuned_weights.pkl', 'wb')
# pickle.dump(bdtClf, outClfs)
# outClfs.close()

In [ ]:
#Unpickles classifier

# outClfs = open('svmClf_untuned_noweights.pkl', 'rb')
# svmClf = pickle.load(outClfs)
# outClfs.close()

In [ ]:
%%time
#Provides classification reports 
if weights == True:    
    bdtScore = bdtClf.fit(xTrain, yTrain, sample_weight = lumiTrain).decision_function(xTest)
else:
    bdtScore = bdtClf.fit(xTrain,yTrain).decision_function(xTest)
    
bdtPredict = bdtClf.predict(xTest)
print("Classification report for BDT, Tuned, Weights %s:\n%s\n"
      % (bdtClf, metrics.classification_report(yTest, bdtPredict)))

In [ ]:
#Pickles scores

# outfile = open('bdt_classreport_tuned_weights.pkl', 'wb')
# pickle.dump(bdtScore, outfile)
# pickle.dump(bdtPredict, outfile)
# outfile.close()

In [ ]:
#Unpickles scores

# inFile = open('classScores_No_Cross_No_Weights_OddsEvens.pkl', 'rb')
# svmScore = pickle.load(inFile)
# bdtScore = pickle.load(inFile)
# svmPredictor = pickle.load(inFile)
# bdtPredictor = pickle.load(inFile)

# Plots classification results

In [ ]:
%%time
#Plots classification results for signal and background
bdtArrs = []
bdtHists = []

#Separates decision function results into signal and background
#along with training and testing
bdtArrs.append(bdtClf.decision_function(xTrain[yTrain>0.5]).ravel())
bdtArrs.append(bdtClf.decision_function(xTrain[yTrain<0.5]).ravel())
bdtArrs.append(bdtClf.decision_function(xTest[yTest>0.5]).ravel())
bdtArrs.append(bdtClf.decision_function(xTest[yTest<0.5]).ravel())

#Turns those arrays into histograms
bdtHists.append(list(numpy.histogram(bdtArrs[0], normed = True, bins = 40)))
bdtHists.append(list(numpy.histogram(bdtArrs[1], normed = True, bins = 40)))
bdtHists.append(list(numpy.histogram(bdtArrs[2], normed = True, bins = 40)))
bdtHists.append(list(numpy.histogram(bdtArrs[3], normed = True, bins = 40)))

#Defines bin edges, centers, and widths
bdtMax = max([hist[0].max() for hist in bdtHists])*1.2
bdtMin = max([hist[0].min() for hist in bdtHists])
bdtEdges = bdtHists[0][1]
bdtCenters = (bdtEdges[:-1] + bdtEdges[1:])/2.
bdtWidths = (bdtEdges[1:] - bdtEdges[:-1])

In [ ]:
#Normalizes histogram based on maximum value
bdtNormVal1 = max(max(bdtHists[0][0]), max(bdtHists[1][0]))
bdtNormVal2 = max(max(bdtHists[2][0]), max(bdtHists[3][0]))
bdtHists[0][0] = [x/bdtNormVal1 for x in bdtHists[0][0]]
bdtHists[1][0] = [x/bdtNormVal1 for x in bdtHists[1][0]]
bdtHists[2][0] = [x/bdtNormVal2 for x in bdtHists[2][0]]
bdtHists[3][0] = [x/bdtNormVal2 for x in bdtHists[3][0]]

In [ ]:
%%time
#Plots histograms
ax1 = plt.subplot(111)
ax1.bar(bdtCenters-bdtWidths/2.,bdtHists[0][0],facecolor='red',linewidth=0,width=bdtWidths,label='Signal',alpha=0.5)
ax1.bar(bdtCenters-bdtWidths/2.,bdtHists[1][0],facecolor='blue',linewidth=0,width=bdtWidths,label='Background',alpha=0.5)
#Change depending on which classifier and options are chosen
plt.title("Classification, BDT, Tuned, Weights, Rand, 15 feats, Training Set")
plt.xlabel("classifier score")
plt.ylabel("Counts/Bin")
legend = ax1.legend(loc='upper center', shadow=True,ncol=2)
for alabel in legend.get_texts():
            alabel.set_fontsize('small')
plt.legend(loc='upper left')
plt.show()
 
ax2 = plt.subplot(111)
ax2.bar(bdtCenters-bdtWidths/2.,bdtHists[2][0],facecolor='red',linewidth=0,width=bdtWidths,label='Signal',alpha=0.5)
ax2.bar(bdtCenters-bdtWidths/2.,bdtHists[3][0],facecolor='blue',linewidth=0,width=bdtWidths,label='Background',alpha=0.5)
plt.title("Classification, BDT, Tuned, Weights, Rand, 15 feats, Testing Set")
plt.xlabel("classifier score")
plt.ylabel("Counts/Bin")
legend = ax1.legend(loc='upper center', shadow=True,ncol=2)
for alabel in legend.get_texts():
            alabel.set_fontsize('small')
plt.legend(loc='upper left')
plt.show()

In [ ]:
%%time
#Plots roc curve, code taken from sklearn website
fpr, tpr, _ = roc_curve(yTest, bdtScore)
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2;
plt.plot(fpr, tpr, color='darkorange',
        lw = lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw = lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC BDT, Tuned, Weights, Rand, 15 feats')
plt.legend(loc="lower right")
plt.show()

In [ ]:
%%time
#Plots confusion matrix, code taken from sklearn
classNames = ['Background','Signal']
confMat = confusion_matrix(yTest, bdtPredict)

#Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(confMat, classes=classNames,
                      title='Confusion matrix, BDT, Tuned, Un-normalization, Weights')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(confMat, classes=classNames, normalize=True,
                      title='Confusion Matrix, BDT, Tuned, Normalized, Weights')

plt.show()

In [ ]:
%%time
#Calculates Matthews Correlation Coefficient
#Ranges from -1 to 1, with 1 being a perfect predictor
matthews_corrcoef(yTest,bdtPredict)